In [5]:
from tqdm import tqdm

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import natsort
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Flatten, Dropout
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator
from sklearn import model_selection
from keras.models import load_model
from sklearn.preprocessing import LabelBinarizer
from keras.callbacks import ModelCheckpoint
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [6]:
import glob

df = pd.read_csv('data/train.csv')
test_data = pd.read_csv('data/test.csv')
submission = pd.read_csv('data/sample_submission.csv')
df.head()

path="data/"
im_file=os.listdir(path)
im_file=natsort.natsorted(im_file)

,id,label
0,1,9
1,2,0
2,3,0
3,4,3
4,5,0


In [7]:
TRAIN_DIR = 'train'
labels=[]

IMG_SIZE=28
data=[]
for i in range(30000):
    path = os.path.join(TRAIN_DIR,os.listdir(TRAIN_DIR)[i])
    img_data = cv2.imread(path)
    img_data = cv2.resize(img_data, (IMG_SIZE,IMG_SIZE))
    image=img_to_array(img_data)
    data.append(image)
    number = int(os.listdir(TRAIN_DIR)[i][:-4])  
    categ_num = int(df[df.id==number].label)
    labels.append(categ_num)

KeyboardInterrupt: 

In [ ]:
num_classes=len(np.unique(labels))
data=np.array(data,dtype="float32")/255.0
labels=np.array(labels)

In [1]:
labels

NameError: name 'labels' is not defined